In [1]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
# 20,000 top most frequent words to consider.
# Any less frequent word will appear as 0 in the sequence data.
# 0 is used as UNKNOWN.
max_features = 20000
# Each review can have variable number of words, in them
# we will consider first 80 words to predict sentence
# if review is shorter we'll pad them by 0.
maxlen = 80
# batch size for gradient learning.
batch_size = 32

In [3]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [4]:
# each word is encoded using word vector
# x_train[0] contains the index of particualr word in word2vec
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
# truncate longer sequence
# padd with zero for shorter sequence
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [6]:
print('Build model...')
# sequential model, we will stack layers sequentially
model = Sequential()
# the Embedding layer is a simple matrix multiplication that transforms words into their corresponding word embeddings.
# (nb_words, vocab_size) x (vocab_size, embedding_dim) = (nb_words, embedding_dim)
# One hot encoded vectos in vocab size x each word in embedding dim = input
model.add(Embedding(max_features, 128))
# 128 LSTM units with tanh activation
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# Just single regular densely-connected NN layer as output.
# output = sigmoid(Wx+b)
model.add(Dense(1, activation='sigmoid'))

Build model...


In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [12]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 141s - loss: 0.2947 - acc: 0.8804 - val_loss: 0.3880 - val_acc: 0.8278
Epoch 2/15
25000/25000 [==============================] - 141s - loss: 0.2137 - acc: 0.9190 - val_loss: 0.4064 - val_acc: 0.8297
Epoch 3/15
25000/25000 [==============================] - 140s - loss: 0.1535 - acc: 0.9432 - val_loss: 0.4551 - val_acc: 0.8298
Epoch 4/15
25000/25000 [==============================] - 141s - loss: 0.1070 - acc: 0.9617 - val_loss: 0.5234 - val_acc: 0.8255
Epoch 5/15
25000/25000 [==============================] - 139s - loss: 0.0837 - acc: 0.9702 - val_loss: 0.5935 - val_acc: 0.8217
Epoch 6/15
25000/25000 [==============================] - 139s - loss: 0.0557 - acc: 0.9826 - val_loss: 0.7143 - val_acc: 0.8207
Epoch 7/15
25000/25000 [==============================] - 140s - loss: 0.0418 - acc: 0.9865 - val_loss: 0.7440 - val_acc: 0.8222
Epoch 8/15
25000/25000 [==============

In [14]:
print(score, acc)

1.20720062368 0.80904


In [15]:
model.save("lstm_sentiment_model.h5")

In [16]:
from keras.layers import GRU

model2 = Sequential()
model2.add(Embedding(max_features, 128))
model2.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))

In [17]:
model2.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

In [21]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 2,658,817
Trainable params: 2,658,817
Non-trainable params: 0
_________________________________________________________________


In [18]:
print('Train...')
model2.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model2.evaluate(x_test, y_test,
                            batch_size=batch_size)

Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 135s - loss: 0.4679 - acc: 0.7727 - val_loss: 0.3769 - val_acc: 0.8362
Epoch 2/15
25000/25000 [==============================] - 135s - loss: 0.2983 - acc: 0.8776 - val_loss: 0.3551 - val_acc: 0.8444
Epoch 3/15
25000/25000 [==============================] - 135s - loss: 0.1865 - acc: 0.9293 - val_loss: 0.4466 - val_acc: 0.8291
Epoch 4/15
25000/25000 [==============================] - 135s - loss: 0.1133 - acc: 0.9602 - val_loss: 0.4976 - val_acc: 0.8242
Epoch 5/15
25000/25000 [==============================] - 136s - loss: 0.0717 - acc: 0.9754 - val_loss: 0.6008 - val_acc: 0.8223
Epoch 6/15
25000/25000 [==============================] - 136s - loss: 0.0457 - acc: 0.9846 - val_loss: 0.7898 - val_acc: 0.8205
Epoch 7/15
25000/25000 [==============================] - 135s - loss: 0.0322 - acc: 0.9894 - val_loss: 0.7794 - val_acc: 0.8129
Epoch 8/15
25000/25000 [==============

In [19]:
print('')
print(score, acc)


1.28998386906 0.80468


GRU does well similar to LSTM

In [20]:
model2.save("gru_sentiment_model.h5")

The h5py package is a Pythonic interface to the HDF5 binary data format.<br/>

It lets you store huge amounts of numerical data, and easily manipulate that data from NumPy. For example, you can slice into multi-terabyte datasets stored on disk, as if they were real NumPy arrays. Thousands of datasets can be stored in a single file, categorized and tagged however you want.

We save models in h5 format, basically storing weights of neural networks.